In [14]:
#Add word count field to discussion forum document in mongodb

from pymongo import MongoClient
from pymongo.errors import BulkWriteError

client = MongoClient()
dbname = 'crime101x2014'
db = client[dbname]

def calculate_wordcount(textcontent):
    words = ''.join(c if c.isalnum() else ' ' for c in textcontent).split()
    return len(words)

def process_forumposts(db):
    # Find all forum posts
    cursor = db.discussion_forum.find()
    
    bulk_op = db.discussion_forum.initialize_unordered_bulk_op()

    for document in cursor:
        #print document
        mongo_id = document['_id']
        body = document['body']
        title = ""
        if 'title' in document: 
            title = document['title']
        forumpostcontent = body + " " + title
        wordcount = calculate_wordcount(forumpostcontent)
        
        bulk_op.find({'_id': mongo_id}).update({'$set': {'wordcount': wordcount}})
        
    try:
        bulk_op.execute()
    except BulkWriteError as bwe:
        print bwe.details

        
        
        
        

In [15]:
process_forumposts(db)